In [1]:
import re
import json
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
import sqlite3
from sqlalchemy import create_engine
from sqlalchemy.types import FLOAT, INTEGER, VARCHAR, TEXT

In [3]:
engine = create_engine('sqlite:///epl_df.db')

In [4]:
# URL list for request loop
url_list = ['stats', 'keepers', 'shooting', 'passing_types', 'gca', 
            'defense', 'possession', 'misc']

for table in url_list:
    url = ('https://fbref.com/en/comps/9/10728/' + table + 
           '/2020-2021-Premier-League-Stats')
    res = requests.get(url)
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("",res.text),'lxml')
    
    soup_a = soup.find_all('thead')
    soup_b = soup_a[2].find_all('tr')[1] #for column names
    soup_c = soup.find_all('tbody')[2] #for data
    
    #create column headers 
    column_list = []
    for element in soup_b:
        if element.string != '\n':
            if element.string.lower() in column_list:
                column_list.append(element.string.lower() + '_2')
            else:
                column_list.append(element.string.lower())
        else:
            pass
        
    # place every soup element into a list
    clean_list = []
    for element in soup_c:
        if element != '\n':
            clean_list.append(element)
            
    #create main df for table creation
    main_stats = pd.DataFrame(columns=column_list)
    
    # special case extraction for the main_stats table
    if table == 'stats':
        for idx in range((len(clean_list))):
            player_stats = []
            for element in clean_list[idx]:
                if element.string == None:
                    nat = element.select('span')[0].contents[1].string.strip()
                    player_stats.append(nat)
                else:
                    player_stats.append(element.string)
            try:
                main_stats.loc[len(main_stats)] = player_stats
            except ValueError:
                pass
            
    # extraction for all other tables        
    else:
        for idx in range((len(clean_list))):
            player_stats = []
            for element in clean_list[idx]:
                player_stats.append(element.string)
            try:
                main_stats.loc[len(main_stats)] = player_stats
            except ValueError:
                pass       
    
    df_sql = main_stats.drop_duplicates(subset ='player', keep = 'first', 
                                        inplace = True)
    
    # choose features of interest and save to a db table
    
    # create main_stats table
    if table == 'stats':
        df_sql = main_stats[['player', 'nation', 'pos', 'squad', 'age', 'mp',
                             'starts', 'min', 'gls', 'ast', 
                             'crdr','crdy']]
        df_sql.to_sql(name='main_stats', con=engine, if_exists='replace', 
                        index=False,
                        dtype={'player': VARCHAR(length=255),
                               'nation': VARCHAR(length=255),
                               'pos': VARCHAR(length=255),
                               'squad': VARCHAR(length=255),
                               'age': INTEGER,
                               'mp': INTEGER,
                               'starts': INTEGER,
                               'min': INTEGER,
                               'gls': INTEGER,
                               'ast': INTEGER,
                               'crdr': INTEGER,
                               'crdy': INTEGER})
    
    # create keepers table
    elif table == 'keepers':
        df_sql = main_stats[['player', 'cs', 'pksv', 'saves']]
        df_sql.to_sql(name='keepers', con=engine, if_exists='replace', 
                      index=False,
                      dtype={'player': VARCHAR(length=255),
                             'cs': INTEGER,
                             'pksv': INTEGER,
                             'saves': INTEGER})
    # create shooting table
    elif table == 'shooting':
        df_sql = main_stats[['player', 'fk', 'pk', 'sh', 'sot']]
        df_sql.to_sql(name='shooting', con=engine, if_exists='replace',
                      index=False,
                      dtype={'player': VARCHAR(length=255),
                             'fk': INTEGER,
                             'pk': INTEGER,
                             'sh': INTEGER,
                             'sot': INTEGER})
    # create passing_types table
    elif table == 'passing_types':
        df_sql = main_stats[['player', 'ck', 'cmp', 'crs', 'ti']]
        df_sql.to_sql(name='passing_types', con=engine, if_exists='replace',
                      index=False,
                      dtype={'player': VARCHAR(length=255),
                             'ck': INTEGER,
                             'cmp': INTEGER,
                             'crs': INTEGER,
                             'ti': INTEGER,})
    # create gca table
    elif table == 'gca':
        df_sql = main_stats[['player', 'gca', 'sca']]
        df_sql.to_sql(name='gca', con=engine, if_exists='replace',
                      index=False,
                      dtype={'player': VARCHAR(length=255),
                             'gca': INTEGER,
                             'sca': INTEGER})
    # create defense table
    elif table == 'defense':
        df_sql = main_stats[['player', 'clr', 'int', 'tkl', 'blocks', 
                             'press']]
        df_sql.to_sql(name='defense', con=engine, if_exists='replace',
                      index=False,
                      dtype={'player': VARCHAR(length=255),
                             'clr': INTEGER,
                             'int': INTEGER,
                             'tkl': INTEGER,
                             'blocks': INTEGER,
                             'press': INTEGER})
    # create possession table
    elif table == 'possession':
        df_sql = main_stats[['player', 'carries', 'rec', 'succ', 'touches']]
        df_sql.to_sql(name='possession', con=engine, if_exists='replace',
                      index=False,
                      dtype={'player': VARCHAR(length=255),
                             'carries': INTEGER,
                             'rec': INTEGER,
                             'succ': INTEGER,
                             'touches': INTEGER,})
    # create misc table
    elif table == 'misc':
        df_sql = main_stats[['player', 'fls', 'off', 'og', 'recov',
                             'won']]
        df_sql.to_sql(name='misc', con=engine, if_exists='replace',
                      index=False,
                      dtype={'player': VARCHAR(length=255),
                             'fls': INTEGER,
                             'off': INTEGER,
                             'og': INTEGER,
                             'recov': INTEGER,
                             'won': INTEGER,})
    else:
        pass

In [5]:
#merge all 8 table from fbref into the complete table
with engine.connect() as conn:
    df_main = pd.read_sql("""
    SELECT ms.player, ms.nation, ms.pos, ms.squad, ms.age, ms.starts,
    ms.min, ms.gls, ms.ast, ms.crdr, ms.crdy, pt.ck, pt.cmp, pt.crs, pt.ti, 
    gca.gca, gca.sca, d.clr, d.int, d.tkl, d.blocks, d.press, sh.fk,
    sh.pk, sh.sh, sh.sot, p.carries, p.rec, p.succ, p.touches, misc.fls,
    misc.off, misc.og, misc.recov, misc.won, gk.cs, gk.pksv, gk.saves
    
    FROM main_stats as ms
    LEFT JOIN passing_types as pt
    ON ms.player = pt.player
    LEFT JOIN gca
    ON ms.player = gca.player
    LEFT JOIN defense as d
    ON ms.player = d.player
    LEFT JOIN shooting as sh
    ON ms.player = sh.player
    LEFT JOIN possession as p
    ON ms.player = p.player
    LEFT JOIN misc as misc
    ON ms.player = misc.player
    LEFT JOIN keepers as gk
    ON ms.player = gk.player
    """, conn)
    
# fill nan values with zero
df_main.fillna(0, inplace=True)

# save as the complete table in main database
df_main.to_sql(name='complete', con=engine, if_exists='replace', index=False)

In [6]:
# check table names

from sqlalchemy.engine.reflection import Inspector

inspector = Inspector.from_engine(engine)
inspector.get_table_names()

['complete',
 'defense',
 'gca',
 'keepers',
 'main_stats',
 'misc',
 'passing_types',
 'possession',
 'shooting']

In [7]:
# check contents of the complete table
df = pd.read_sql('select * from complete', engine)
df

,player,nation,pos,squad,age,starts,min,gls,ast,crdr,...,succ,touches,fls,off,og,recov,won,cs,pksv,saves
0,Patrick van Aanholt,NED,DF,Crystal Palace,29,20,"1,777",0,1,0,...,9,1323,11,3,0,179,10,0.0,0.0,0.0
1,Tammy Abraham,ENG,FW,Chelsea,22,12,"1,040",6,1,0,...,6,356,22,5,0,47,39,0.0,0.0,0.0
2,Che Adams,SCO,FW,Southampton,24,30,"2,667",9,5,0,...,25,976,29,19,0,119,47,0.0,0.0,0.0
3,Tosin Adarabioyo,ENG,DF,Fulham,22,33,"2,953",0,0,0,...,8,2221,16,2,1,278,100,0.0,0.0,0.0
4,Adrián,ESP,GK,Liverpool,33,3,270,0,0,0,...,0,113,0,0,0,20,0,1.0,0.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,Andi Zeqiri,SUI,"FW,DF",Brighton,21,0,171,0,0,0,...,1,65,2,0,0,12,2,0.0,0.0,0.0
520,Oleksandr Zinchenko,UKR,DF,Manchester City,23,15,"1,478",0,0,0,...,5,1623,9,1,0,153,24,0.0,0.0,0.0
521,Hakim Ziyech,MAR,"FW,MF",Chelsea,27,15,"1,172",2,3,0,...,18,873,19,4,0,94,3,0.0,0.0,0.0
522,Kurt Zouma,FRA,DF,Chelsea,25,22,"2,029",5,0,0,...,5,1956,17,0,0,210,97,0.0,0.0,0.0
